In [10]:
import numpy as np
import scipy
%matplotlib notebook
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1.1 Tracking by assignment without particle merging or division

### f) Linear program definition
In order to write down the matrix of constraints, you may find convenient to finish implementing the following function:

In [2]:
### Convert edge list to incidence matrix:
def get_incidence_matrix(edge_list, nb_nodes=None):
    """
    Utility function converting a list of uv-edges into an incidence matrix.
    
    edge_list should be a numpy array of shape (number_of_edges, 2) including the uv-pairs 
    for each edge in the graph
    """
    nb_nodes = edge_list.max() + 1 if nb_nodes is None else nb_nodes
    inc_matrix = np.zeros((nb_nodes, edge_list.shape[0]), dtype='int')
    for i, edge in enumerate(edge_list):
        u, v = edge
        inc_matrix[u,i] = -1
        inc_matrix[v,i] = 1
        
    return inc_matrix


Find the constraint matrix `A_eq`:

In [3]:
# First of all, we define the edge list (as uv pairs) of edges in the directed graph 
# of Fig. 2 in the solution sheet. In each row, the third element represents the edge cost:
edge_list = np.array([
    [0,1,-10],
    [0,2,-20],
    [0,3,-18],
    [1,4,5],
    [2,4,16],
    [3,4,41],
    [1,5,26],
    [2,5,5],
    [3,5,20],
    [4,7,-13],
    [5,8,-8],
    [7,6,17],
    [8,6,34],
    [6,9,-9]
]
)


In [7]:
edges = edge_list[:,:2]
costs = edge_list[:,2]

# We delete the first and last rows, since they are the ones associated to the source (node 0) 
# and target (node 9):
A_eq = get_incidence_matrix(edges)[1:-1]
b_eq = np.zeros(A_eq.shape[0])
print("Constraint matrix A_eq:\n")
print(A_eq)

Constraint matrix A_eq:

[[ 1  0  0 -1  0  0 -1  0  0  0  0  0  0  0]
 [ 0  1  0  0 -1  0  0 -1  0  0  0  0  0  0]
 [ 0  0  1  0  0 -1  0  0 -1  0  0  0  0  0]
 [ 0  0  0  1  1  1  0  0  0 -1  0  0  0  0]
 [ 0  0  0  0  0  0  1  1  1  0 -1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  1 -1]
 [ 0  0  0  0  0  0  0  0  0  1  0 -1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  0 -1  0]]


### g) Solve the linear program
To solve the LP you may find useful the following wrapper of `scipy.optimize.linprog`.

In [11]:
from scipy.optimize import linprog

def solve_LP(costs, A_eq=None, b_eq=None, A_leq=None, b_leq=None, bounds=(0, None), 
            edge_list=None):
    """
    A wrapper around `scipy.optimize.linprog`.
    
    The `bounds` parameter represents what in the exercise sheet is defined as (x_low, x_high)
    """
    optim_result = scipy.optimize.linprog(costs, A_ub=A_leq, b_ub=b_leq, A_eq=A_eq, 
                                          b_eq=b_eq,  bounds=bounds, method='revised simplex')
    solution = optim_result.x
    assert optim_result.status == 0, "Something went wrong during the optimization"
    
    # Do some printing:
    np.set_printoptions(precision=4)
    print("LP solution: \n", solution)
    print("LP minimum energy: ", optim_result.fun)
    
    # Print selected edges:
    if edge_list is not None:
        assert edge_list.shape[0] == solution.shape[0]
        for i, edge in enumerate(edge_list):
            if np.allclose(solution[i], 1.):
                print("Edge ({},{}) selected".format(edge[0], edge[1]))
    
    return solution, optim_result.fun

In [12]:
### Your solution goes here
solution, energy = solve_LP(costs, A_eq=A_eq, b_eq=b_eq, bounds=(0,1), edge_list=edges)

LP solution: 
 [1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1.]
LP minimum energy:  -10.0
Edge (0,1) selected
Edge (1,4) selected
Edge (4,7) selected
Edge (7,6) selected
Edge (6,9) selected


### h) Particle appearance and disappearance

In [14]:
# Let's define the additional appearance and disappearance edges:
disapperance_edges = np.array([
    [1,9,10],
    [2,9,10],
    [3,9,10],
    [7,9,10],
    [8,9,10],
])

appearance_edges = np.array([
    [0,4,6],
    [0,5,6],
    [0,6,6],    
])

In [15]:
# And now we solve again the LP with the additional edges:
edge_info = np.concatenate((edge_list, appearance_edges, disapperance_edges), axis=0)
edges = edge_info[:,:2]
costs = edge_info[:,2]
A_eq = get_incidence_matrix(edges)[1:-1]
print("Constraint matrix A_eq:\n")
print(A_eq)
b_eq = np.zeros(A_eq.shape[0])

solution, energy = solve_LP(costs, A_eq=A_eq, b_eq=b_eq, bounds=(0,1), edge_list=edges)

Constraint matrix A_eq:

[[ 1  0  0 -1  0  0 -1  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0]
 [ 0  1  0  0 -1  0  0 -1  0  0  0  0  0  0  0  0  0  0 -1  0  0  0]
 [ 0  0  1  0  0 -1  0  0 -1  0  0  0  0  0  0  0  0  0  0 -1  0  0]
 [ 0  0  0  1  1  1  0  0  0 -1  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  1  1  0 -1  0  0  0  0  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  1 -1  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1  0 -1  0  0  0  0  0  0  0  0 -1  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  0 -1  0  0  0  0  0  0  0  0 -1]]
LP solution: 
 [1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1.]
LP minimum energy:  -32.0
Edge (0,1) selected
Edge (0,2) selected
Edge (0,3) selected
Edge (1,4) selected
Edge (2,5) selected
Edge (4,7) selected
Edge (5,8) selected
Edge (6,9) selected
Edge (0,6) selected
Edge (3,9) selected
Edge (7,9) selected
Edge (8,9) selected


# 1.2 Bonus: Tracking with particle merging and/or division

### e) Non integral solution
Below you can find two examples of cost vectors: the first one gives a non-integral solution, wherease the second one gives an integral solution.

In [25]:
A_eq = np.array([
    [1, 1, -1, -1],
    [1,-1,  0,  0]
])
b_eq = np.array([0, 0])

# Example with non-integral solution:
costs = np.array([2, 5, -3, -10])
print("Example with NON-INTEGRAL solution:")
solution, energy = solve_LP(costs, A_eq=A_eq, b_eq=b_eq, bounds=(0,1))

print("\nExample with INTEGRAL solution:")
costs = np.array([1, 5, -19, -20])
solution, energy = solve_LP(costs, A_eq=A_eq, b_eq=b_eq, bounds=(0,1))

Example with NON-INTEGRAL solution:
LP solution: 
 [0.5 0.5 0.  1. ]
LP minimum energy:  -6.5

Example with INTEGRAL solution:
LP solution: 
 [1. 1. 1. 1.]
LP minimum energy:  -33.0
